# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [1]:
# Imports
import pandas as pd
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import string
import re
import numpy as np
import math
import random
import regex

# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [2]:
#TODO: implement!
url = 'https://raw.githubusercontent.com/alch00001/NNTI-WS2021-NLP-Project/main/data/hindi_hatespeech.tsv'
data = pd.read_csv(url, sep='\t')

#Development set
development = data.loc[0:99].copy()
development.astype("string")
development.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NOT,NONE,NONE
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,HOF,PRFN,TIN
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",NOT,NONE,NONE
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,NOT,NONE,NONE


In [3]:
hate = 0
no_hate = 0
for i in range(len(development['task_1'])):
  if development['task_1'][i] == 'HOF':
    hate += 1
  else:
    no_hate += 1
print(hate, no_hate)


22 78


## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [4]:

development['text'] = development['text'].apply(lambda x:' '.join(x.lower() for x in x.split()))
development['text'] = development['text'].apply(lambda x: regex.sub(r'(#[^\s]*)*', '',x))     
development['text'] = development['text'].apply(lambda x: regex.sub(r'(@[\w]*)*[\d~\|\p{Punct}*]*(http[^\s]*)*', '',x)) 
development['text'] = development['text'].apply(lambda x: regex.sub(r'<[^<]+?>','',x)) #remove html 
development['text'] = development['text'].apply(lambda x: regex.sub(r'href=','',x)) 
development['text'] = development['text'].apply(lambda x: x.lower())                                                          #make lower case
#import and remove stopwords
stopurl = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-hi/master/stopwords-hi.txt'
stopwords = pd.read_csv(stopurl, sep='\t', header=None)
development['text'] = development['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords[:][0].tolist()))
#remove emojis, this script was taken from github
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
development['text'] = development['text'].apply(lambda x: remove_emoji(x))

In [5]:
print(development["text"].head())

0                 बांग्लादेश शानदार वापसी भारत रन रोका
1          सब रंडी नाच देखने व्यस्त होगा सब शुरू देंगे
2    तुम हरामियों बस जूतों कमी शुक्र तुम्हारी लिंचि...
3    बीजेपी mla आकाश विजयवर्गीय जेल रिहा जमानत मिलन...
4    चमकी बुखार विधानसभा परिसर आरजेडी प्रदर्शन तेजस...
Name: text, dtype: object


## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output od the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [6]:
#TODO: implement!
#this is only unique words
V = list(set(development['text'].str.split(' ').sum()))
if len(V[0]) == 0: #resolve an issue with emptry string appearing in vocab
  V.pop(0)
corpus = ' '.join([i for i in development['text']]).split()
#all words given assigned a unique value - this line doesn't work
#unique_val = list(pd.factorize(corpus))
word2idx = {w: idx for (idx, w) in enumerate(V)}
idx2word = {idx: w for (idx, w) in enumerate(V)}
print(len(V))
print(len(corpus))

1110
1537
['बोलेंगे', 'बहन', 'व्यस्तता', 'नहाने', 'साला', 'hsp', 'उलझन', 'जुनून', 'श्रद्धांजलि', 'समय']


* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [7]:
#TODO: implement!

def word_to_one_hot(word):
  one_hot = [0 if word != x else 1 for x in V]
  return one_hot

X = word_to_one_hot(V[1])
len(X)
        

1110

## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

In [8]:
#TODO: implement!
def word_frequency(word):
  freq = 0
  for x in corpus:
    if x == word:
      freq += 1
  return freq

def sampling_prob(word):
  relative_frq = word_frequency(word)/len(V)
  if relative_frq==0:          #if word is not present in the corpus
    return 0
  else:
    p_keep = (np.sqrt(relative_frq/0.000001)+1)*(0.000001/relative_frq)
    return p_keep

sampling_prob('भारत')

0.015121664425751339

# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [9]:
#TODO: implement!
window_size = 5

def get_target_context(sentence):
  words = sentence.split()
  training_data = []
  for word in words:
   # print("Before subsampling the size of words is :", len(words))
    if random.random() < sampling_prob(word): 
      words.remove(word)
  #print("after subsampling the size of words is :", len(words))
  for word in words:
    #context = []
    position = words.index(word) 
    for i in range(-window_size, window_size+1):
      if position+i<0 or position+i>=len(words) or i==0:
        continue
      #print(context)
      training_data.append([word, words[position+i]])
  #print("Final size of words is :", len(words))
  yield training_data


# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [10]:
# Set hyperparameters
window_size = 5
embedding_size = 300 
vocab_size = len(V)
batch_size = 10
learning_rate = 0.001
epochs = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [11]:
#Create Model

class Word2Vec(nn.Module):
  def __init__(self):
    super(Word2Vec, self).__init__()
    # Hidden layer
    self.l1 = nn.Linear(len(V), embedding_size)
    # Output layer
    self.l2 = nn.Linear(embedding_size, len(V))

  def forward(self, one_hot):
    Z1 = self.l1(one_hot)
    out = self.l2(Z1)
    return out
    #return F.log_softmax(out, dim=0) - we don't need this line because nn.crossentropy() combine nlllLoss and softmax


# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [12]:
# Define optimizer and loss
model = Word2Vec()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,)
loss_func = nn.CrossEntropyLoss()
model = model.to(device=device)

#we don't really need to initialize weights because it will be done automatically with Pytorch
#but we are following the instructions
nn.init.uniform_(model.l1.weight, -.5,.5) #initialize our weights to small values between -.5 and .5 
nn.init.uniform_(model.l2.weight, -.5,.5)

w1 = model.l1.weight
w2 = model.l2.weight

# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [13]:
#iterate through every sentence and create a list 
#of target,context pairs stored in variable 'pairs'
train_set = development['text'].str.split(',')
train_set = train_set.to_list()
print("sentences :",len(train_set))
train_set = [''.join(x) for x in train_set]
print('train set created')
pairs = []
for sentence in train_set:
  p = get_target_context(sentence)
  pairs += p

print('pairs formed')
#removing an extra pair of braces in this line
flattened = [val for sublist in pairs for val in sublist]


sentences : 100
train set created
pairs formed


In [15]:
from torch.utils.data import Dataset, DataLoader
#create our dataloader to iterate thorugh our pairs
dataloader = DataLoader(
    flattened, 
    batch_size=batch_size)

def train():
  threshold = .115 #stopping criterion
  prev_loss = 0
  for epoch in range(epochs):
    total_loss = 0
    for i,(targets, contexts)in enumerate(dataloader): #returns a list of word,context pairs in string format of length batch
        #convert target/center words to one hot encodings
        word = torch.FloatTensor([word_to_one_hot(x) for x in targets])
        output = model(word.to(device=device))
        #convert context to appropriate format since loss function implicitly will create
        #one hot encodings from class values. 
        context = torch.tensor([word2idx[w] for w in contexts], dtype=torch.long)
        context = context.to(device=device)
        #print(context.shape)
        loss = loss_func(output, context)

        # Backprop to update model parameters 
        total_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Total loss at epoch: ",epoch+1, "loss: ", total_loss/(len(dataloader)), "loss comparison: ",abs(prev_loss-total_loss/(len(dataloader))))
    if abs(prev_loss-total_loss/(len(dataloader))) <= threshold:
      break
    prev_loss = total_loss/(len(dataloader))
  print("Training finished")





In [16]:
train()
w1 = model.l1.weight
print(w1.shape)

Total loss at epoch:  1 loss:  tensor(7.9382, device='cuda:0', grad_fn=<DivBackward0>) loss comparison:  tensor(7.9382, device='cuda:0', grad_fn=<AbsBackward>)
Total loss at epoch:  2 loss:  tensor(6.5391, device='cuda:0', grad_fn=<DivBackward0>) loss comparison:  tensor(1.3991, device='cuda:0', grad_fn=<AbsBackward>)
Total loss at epoch:  3 loss:  tensor(5.3043, device='cuda:0', grad_fn=<DivBackward0>) loss comparison:  tensor(1.2349, device='cuda:0', grad_fn=<AbsBackward>)
Total loss at epoch:  4 loss:  tensor(4.2876, device='cuda:0', grad_fn=<DivBackward0>) loss comparison:  tensor(1.0167, device='cuda:0', grad_fn=<AbsBackward>)
Total loss at epoch:  5 loss:  tensor(3.5559, device='cuda:0', grad_fn=<DivBackward0>) loss comparison:  tensor(0.7317, device='cuda:0', grad_fn=<AbsBackward>)
Total loss at epoch:  6 loss:  tensor(3.1364, device='cuda:0', grad_fn=<DivBackward0>) loss comparison:  tensor(0.4195, device='cuda:0', grad_fn=<AbsBackward>)
Total loss at epoch:  7 loss:  tensor(2.

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).

In [17]:
#Saving Model

#from google.colab import drive
#drive.mount('/content/gdrive')
model_save_name = 'classifier.pth'
#path = F"/content/gdrive/My Drive/{model_save_name}" 
#torch.save(model.state_dict(), path)

torch.save(model.state_dict(), 'checkpoint.pth') #saving weights
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())
print(state_dict['l2.weight'].shape)
weights = state_dict['l1.weight']



odict_keys(['l1.weight', 'l1.bias', 'l2.weight', 'l2.bias'])
torch.Size([1110, 300])


In [18]:
#create dictionary mapping vocab words to tensors
weights = torch.transpose(weights,0,1)
vectors = {}
w = weights.cpu()
for i in range(len(V)):
  vectors[V[i]] = w[i].numpy()


In [19]:
#Write these dictionary to text file in the format:
# word tensorval1 tensorval2 ....tensorval2
from itertools import chain
import io
with open(r'C:\Users\kusha\Desktop\BasicEmbeddings.txt', 'w', encoding="utf-8") as f:
  f.write("\n".join(" ".join(chain([key],[str(number) for number in value])) for key,value in vectors.items()))  
f.close()   

In [20]:
!nvidia-smi

Tue Mar 30 18:28:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W /  70W |   1076MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
torch.cuda.get_device_name()

'Tesla T4'